In [24]:
!pip install pysoundfile

In [0]:
!rm -rf RAVDESS/split_image     # use this to clean the Data Set folder
#!rm -rf RAVDESS.zip  # use this to force re download

In [0]:
# helper to download the Data Set from Google Drive

import requests
import os

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


# Download the Data Set

Uncomment the `file_id` of the Data Set you want to use.




In [0]:
# RAVDESS_original.zip https://drive.google.com/open?id=1WyJsDuxJlUObBCFrNZLAXM2w4-sp0CxN
# file_id = '1WyJsDuxJlUObBCFrNZLAXM2w4-sp0CxN'

# RAVDESS_trimmed.zip https://drive.google.com/open?id=1pFum_YGf2C82HJvMwt0gi8apj6PJLi0s
#file_id = '1pFum_YGf2C82HJvMwt0gi8apj6PJLi0s'

# RAVDESS_enriched.zip https://drive.google.com/open?id=1LG42oQTSs6HWMLsdqhFKbADA2wi8234_
file_id = '1LG42oQTSs6HWMLsdqhFKbADA2wi8234_'

# RAVDESS_spectogram.zip https://drive.google.com/open?id=1jDb2GDnapxx-5bhRR4rudVLxd4ajrpF-
# file_id = '1jDb2GDnapxx-5bhRR4rudVLxd4ajrpF-'

destination = 'RAVDESS.zip'

if not os.path.exists(destination):
    download_file_from_google_drive(file_id, destination)

In [28]:
!ls -l

total 1033616
drwxr-xr-x 7 root root       4096 Nov 18 10:40 RAVDESS
-rw-r--r-- 1 root root 1058409423 Nov 18 10:38 RAVDESS.zip
drwxr-xr-x 1 root root       4096 Nov  6 16:17 sample_data


In [29]:
!unzip -n RAVDESS.zip

Archive:  RAVDESS.zip
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-01-00.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-01-01.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-01-02.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-02-00.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-02-01.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-02-02.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-03-00.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-03-01.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-03-02.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-04-00.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-04-01.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-04-02.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-05-00.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-05-01.png  
  inflating: RAVDESS/spectograms/03-01-01-01-01-01-05-02.png  
  inflating: RAVDESS/spectograms/

In [49]:
!ls RAVDESS/ -l

total 380
drwxr-xr-x 2 root root 245760 Oct 15 23:27 enriched
drwxr-xr-x 2 root root  69632 Oct 15 23:28 original
drwxr-xr-x 2 root root  69632 Oct 15 23:28 trimmed


In [50]:
!echo "Files in RAVDESS/original:" $(find RAVDESS/original/ -type f| wc -l)
!echo "Files in RAVDESS/trimmed:" $(find RAVDESS/trimmed/ -type f| wc -l)
!echo "Files in RAVDESS/enriched:" $(find RAVDESS/enriched/ -type f| wc -l)
!echo "Files in RAVDESS/spectograms:" $(find RAVDESS/spectograms/ -type f| wc -l)

Files in RAVDESS/original: 1440
Files in RAVDESS/trimmed: 1440
Files in RAVDESS/enriched: 4320
find: ‘RAVDESS/spectograms/’: No such file or directory
Files in RAVDESS/spectograms: 0


In [0]:
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
from scipy.signal import spectrogram
import math
import matplotlib.pyplot as plt
import random
import re
import gc
from IPython.display import Image, display
from sklearn.model_selection import train_test_split
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [0]:
random.seed(42) # set the random seed

In [0]:
# This class handles the preparation of the RAVDESS Data Set

class RAVDESS:
    
    def __init__(self, frequency):
        # store audio file name identifier mapping helpers, see the original documentation https://zenodo.org/record/1188976#.XaYUupIzaUk
        self.frequency = frequency
        self.separator = "-"
        self.audio_only = "03"
        self.voice_channel = "01"
        self.emotions = {
            "01": "neutral", 
            "02": "calm", 
            "03": "happy", 
            "04": "sad", 
            "05": "angry", 
            "06": "fearful", 
            "07": "disgust", 
            "08": "surprised"
        }
        self.emotion_intensities = {"01": "normal", "02": "strong"}
        self.statements = {"01": "Kids_are_talking_by_the_door", "02": "Dogs_are_sitting_by_the_door"}
        self.repetitions = ["01", "02"]
        self.actors = {
            "01": "male", 
            "02": "female", 
            "03": "male", 
            "04": "female", 
            "05": "male", 
            "06": "female", 
            "07": "male", 
            "08": "female", 
            "09": "male", 
            "10": "female", 
            "11": "male", 
            "12": "female", 
            "13": "male", 
            "14": "female", 
            "15": "male", 
            "16": "female", 
            "17": "male", 
            "18": "female", 
            "19": "male", 
            "20": "female", 
            "21": "male", 
            "22": "female", 
            "23": "male",
            "24": "male",
        }
        self.shown_specs = 0

        # store helper paths
        self.data_set_path = 'RAVDESS/'
        self.original_data_path = self.data_set_path + 'original/'
        self.trimmed_data_path = self.data_set_path + 'trimmed/'
        self.enriched_data_path = self.data_set_path + 'enriched/'
        self.spectograms_data_path = self.data_set_path + 'spectograms/'
        self.split_image_data_path = self.data_set_path + 'split_image/'

        # create the directories if needed
        if not os.path.exists(self.trimmed_data_path):
          os.mkdir(self.trimmed_data_path)

        if not os.path.exists(self.enriched_data_path):
          os.mkdir(self.enriched_data_path)

        if not os.path.exists(self.spectograms_data_path):
          os.mkdir(self.spectograms_data_path)

        if not os.path.exists(self.split_image_data_path):
          os.mkdir(self.split_image_data_path)

        return

    # List all audio file paths
    def get_audio_paths_common(self, directory):
        audio_files = os.listdir(directory) # list all audio files
        audio_files = [directory + audio_file_name for audio_file_name in audio_files] # append prefix path
        return audio_files

    def get_original_audio_paths(self):
        return self.get_audio_paths_common(self.original_data_path)

    def get_trimmed_audio_paths(self):
        return self.get_audio_paths_common(self.trimmed_data_path)

    def get_enriched_audio_paths(self):
        return self.get_audio_paths_common(self.enriched_data_path)
    
    def get_spectograms_paths(self):
        return self.get_audio_paths_common(self.spectograms_data_path)
    
    def get_split_image_paths(self):
        return self.get_audio_paths_common(self.split_image_data_path)

    def get_file_name_without_extension(self, file_path):
        return os.path.splitext(os.path.basename(file_path))[0]

    
    # Trim the silent data from the audio files
    def trim_audios(self):
        print('Trimming audio files - remove the silent data from the audio files')
        num_of_original_files = len(os.listdir(self.original_data_path))
        num_of_trimmed_files = len(os.listdir(self.trimmed_data_path))
        print(str(num_of_trimmed_files) + '/' + str(num_of_original_files))

        audio_paths = self.get_original_audio_paths()
        finished = 0
        for audio_path in audio_paths:
            new_trimmed_file_path = self.trimmed_data_path + os.path.basename(audio_path) # on a new path with the same file name

            # if the trimmed file doesn't exist create it
            if not os.path.exists(new_trimmed_file_path):
                audio_data, samplerate = librosa.load(audio_path, sr=self.frequency) # load and convert the audio
                audio_data_filtered, index = librosa.effects.trim(audio_data, top_db=30, frame_length=2048, hop_length=512) # trim the silent data from the audio file

                librosa.output.write_wav(new_trimmed_file_path, audio_data_filtered, samplerate) # write out the trimmed data

                finished += 1
                print(str(num_of_trimmed_files + finished) + '/' + str(num_of_original_files) + ' Trimmed: ' + audio_path) # log progress
    
    # Add noise to the audio to create more samples
    def create_audio_with_noise(self, data, number_of_noisy_audios):
        mean, std = data.mean(), data.std()
        noised_data = []

        # Add noise based on the standard deviation
        for i in range(number_of_noisy_audios):
            noise = np.random.normal(mean, std*random.uniform(0.1, 0.3), data.shape) 
            noised_data.append(data + noise)
        return noised_data

    # Save the noised audio data
    def create_save_noised_audio(self, number_of_noisy_audios):
        print('Noise audios - enrich the original ' + str(number_of_noisy_audios + 1) + ' times')

        num_of_trimmed_files = len(os.listdir(self.trimmed_data_path))
        num_of_needed_enriched_files = num_of_trimmed_files * (number_of_noisy_audios + 1)
        num_of_enriched_files = len(os.listdir(self.enriched_data_path))
        print(str(num_of_enriched_files) + '/' + str(num_of_needed_enriched_files))

        audio_paths = self.get_trimmed_audio_paths()
        finished = 0
        for audio_path in audio_paths:
            audio_data, samplerate = sf.read(audio_path) # load the audio

            new_path_base = self.enriched_data_path + self.get_file_name_without_extension(audio_path) # set directory and remove file extension from the original path
            unmodified_data_new_path = new_path_base + '-00.wav' # append a new identifier Noised data ID
            if not os.path.exists(unmodified_data_new_path):
                sf.write(unmodified_data_new_path, audio_data, samplerate) # this is the non noised trimmed audio
                finished += 1
                print(str(num_of_enriched_files + finished) + '/' + str(num_of_needed_enriched_files) + ' Enrichment: ' + unmodified_data_new_path) # log progress

            # add noise to the trimmed audio
            noised_data = self.create_audio_with_noise(audio_data, number_of_noisy_audios)

            # write out the noised audio
            for i in range(0, len(noised_data)):
                new_file_name = new_path_base + '-' + "{:02d}".format(i+1) + '.wav'
                if not os.path.exists(new_file_name):
                    sf.write(new_file_name, noised_data[i], samplerate)

                    finished += 1
                    print(str(num_of_enriched_files + finished) + '/' + str(num_of_needed_enriched_files) + ' Enrichment: ' + new_file_name) # log progress
                    
                      
    def create_spectograms(self):
        print('Create spectograms')
        num_of_enriched_files = len(os.listdir(self.enriched_data_path))
        num_of_spectograms = len(os.listdir(self.spectograms_data_path))
        print(str(num_of_spectograms) + '/' + str(num_of_enriched_files))

        audio_paths = self.get_enriched_audio_paths()
        finished = 0
        for audio_path in audio_paths:
            spectogram_path = self.spectograms_data_path + self.get_file_name_without_extension(audio_path) + '.png' # create a path with the same name and a replaced file extension
            if not os.path.exists(spectogram_path):
                audio_data, samplerate = sf.read(audio_path)
                '''window_lenght = samplerate * 1
                window_offset = math.floor(samplerate * 0.1)
                window_start = 0
                split_audio_data = audio_data[window_start : window_start + window_lenght]'''
                nfft = 128
                fig,ax = plt.subplots(1)
                fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
                fig.set_size_inches(8*len(audio_data)/samplerate,8)
                spect, f, t, image = plt.specgram(audio_data, NFFT=nfft, Fs=samplerate, noverlap=nfft/2)
                plt.axis('off')
                plt.gray() # convert the image into black-and-white
                fig.savefig(spectogram_path, dpi=300)

                finished += 1
                print(str(num_of_spectograms + finished) + '/' + str(num_of_enriched_files) + ' Created spectogram: ' + spectogram_path) # log progress

                plt.close()
                #gc.collect() # this won't solve the issue, but it will crash later... https://github.com/matplotlib/matplotlib/issues/8519


    def split_image_data(self, width = 500):
        print('Split image data')
        num_of_spectograms = len(os.listdir(self.spectograms_data_path))
        ls = os.listdir(ravdess.split_image_data_path)
        num_of_splitted_images = len(set([i[:i.rfind('-')] for i in ls]))
        print(str(num_of_splitted_images) + '/' + str(num_of_spectograms))

        spectogram_paths = self.get_spectograms_paths()
        finished = 0
        for spectogram_path in spectogram_paths:
            split_image_path = self.split_image_data_path + self.get_file_name_without_extension(spectogram_path) + '-0.png'
            if not os.path.exists(split_image_path):
                im = Image.open(spectogram_path)
                imgwidth, imgheight = im.size
                split_image_path = self.split_image_data_path + self.get_file_name_without_extension(spectogram_path) + '-'
                for j in range(0,imgwidth,width):
                    if j + width > imgwidth:
                        j = imgwidth - width
                    box = (j, 0, j+width, imgheight)
                    a = im.crop(box)
                    a.save(split_image_path + str(j) + '.png')
                
                finished += 1
                print(str(num_of_splitted_images + finished) + '/' + str(num_of_spectograms) + ' Create split image from: ' + split_image_path )
                
                    
    
    def name_to_data(self, path):
        numbers = re.findall(r'\d+', path) # numbers in the name of the audio file
        # spectogram = Image.open(spectogram_path) # consumes too much memory
        return [
            self.emotions[numbers[2]],
            self.emotion_intensities[numbers[3]],
            self.statements[numbers[4]],
            self.repetitions[int(numbers[5])-1],
            numbers[6], 
            self.actors[numbers[6]], 
            path
        ]
        
    
    # Creates the Data Frame for the RAVDESS
    def create_dataframe(self):
        print('Create dataframe')
        data = []
        specograms_paths = self.get_split_image_paths()
        for specograms_path in specograms_paths:
            data.append(self.name_to_data(specograms_path))

        df = pd.DataFrame(  # contruct DataFrame from data
            { 
                'emotion'           : pd.Categorical([row[0] for row in data]),
                'emotion_intensity' : pd.Categorical([row[1] for row in data]),
                'statement'         : pd.Categorical([row[2] for row in data]),
                'repetition'        : pd.Categorical([row[3] for row in data]),
                'actor'             : pd.Categorical([row[4] for row in data]),
                'actor_gender'      : pd.Categorical([row[5] for row in data]),
                'image'             : pd.Categorical([row[6] for row in data])
            })

        # one-hot encode columns
        df = pd.get_dummies(df, columns=["emotion", "emotion_intensity", "statement", "repetition", "actor", "actor_gender"])
        return df

In [0]:
frequency = 22000
ravdess = RAVDESS(frequency)

In [153]:
# STEP 1: Trim audios
ravdess.trim_audios()

Trimming audio files - remove the silent data from the audio files
1440/1440


In [154]:
# STEP 2: Create noisy audios
ravdess.create_save_noised_audio(number_of_noisy_audios=2)

Noise audios - enrich the original 3 times
4320/4320


In [0]:
# STEP 3: Create spectograms
ravdess.create_spectograms() # might crash due to https://github.com/matplotlib/matplotlib/issues/8519

In [12]:
ravdess.split_image_data()

Split image data
201/1774
202/1774 Create split image from: RAVDESS/split_image/03-01-08-02-01-01-21-00-
203/1774 Create split image from: RAVDESS/split_image/03-01-03-02-01-02-06-00-
204/1774 Create split image from: RAVDESS/split_image/03-01-08-02-01-02-14-02-
205/1774 Create split image from: RAVDESS/split_image/03-01-08-02-01-01-17-02-
206/1774 Create split image from: RAVDESS/split_image/03-01-01-01-01-01-18-00-
207/1774 Create split image from: RAVDESS/split_image/03-01-02-01-02-02-11-01-
208/1774 Create split image from: RAVDESS/split_image/03-01-08-01-02-02-06-00-
209/1774 Create split image from: RAVDESS/split_image/03-01-08-02-01-01-07-02-
210/1774 Create split image from: RAVDESS/split_image/03-01-06-02-02-02-01-00-
211/1774 Create split image from: RAVDESS/split_image/03-01-02-02-02-02-09-00-
212/1774 Create split image from: RAVDESS/split_image/03-01-02-01-01-02-11-00-
213/1774 Create split image from: RAVDESS/split_image/03-01-02-02-02-01-07-01-
214/1774 Create split imag

In [0]:
# Display spectograms

for i in range(10):
  spectograms = ravdess.get_spectograms_paths()
  random_spec =  random.choice(spectograms)
  print(random_spec)
  im = Image.open(random_spec,)
  display(im, )
  # display(Image(random_spec, height=240))

In [60]:
ravdess.get_spectograms_paths()

['RAVDESS/spectograms/03-01-07-02-01-01-24-00.png',
 'RAVDESS/spectograms/03-01-04-02-01-02-11-01.png',
 'RAVDESS/spectograms/03-01-01-01-02-01-06-00.png',
 'RAVDESS/spectograms/03-01-02-01-01-01-01-02.png',
 'RAVDESS/spectograms/03-01-04-02-01-01-10-02.png',
 'RAVDESS/spectograms/03-01-07-01-01-02-16-01.png',
 'RAVDESS/spectograms/03-01-04-02-02-02-22-01.png',
 'RAVDESS/spectograms/03-01-06-02-02-02-05-00.png',
 'RAVDESS/spectograms/03-01-05-02-01-01-12-00.png',
 'RAVDESS/spectograms/03-01-07-02-01-02-01-01.png',
 'RAVDESS/spectograms/03-01-03-02-02-02-05-00.png',
 'RAVDESS/spectograms/03-01-03-01-02-02-13-01.png',
 'RAVDESS/spectograms/03-01-03-02-01-01-12-02.png',
 'RAVDESS/spectograms/03-01-08-02-01-02-05-01.png',
 'RAVDESS/spectograms/03-01-01-01-01-02-08-02.png',
 'RAVDESS/spectograms/03-01-04-01-01-02-22-02.png',
 'RAVDESS/spectograms/03-01-05-02-01-02-05-01.png']

In [0]:
# STEP 4: Create dataframe
df = ravdess.create_dataframe()
df.head()

Create dataframe


,image,emotion_angry,emotion_calm,emotion_disgust,emotion_fearful,emotion_happy,emotion_neutral,emotion_sad,emotion_surprised,emotion_intensity_normal,emotion_intensity_strong,statement_Dogs_are_sitting_by_the_door,statement_Kids_are_talking_by_the_door,repetition_01,repetition_02,actor_01,actor_02,actor_03,actor_04,actor_05,actor_06,actor_07,actor_08,actor_09,actor_10,actor_11,actor_12,actor_13,actor_14,actor_15,actor_16,actor_17,actor_18,actor_19,actor_20,actor_21,actor_22,actor_23,actor_24,actor_gender_female,actor_gender_male
0,RAVDESS/spectograms/03-01-08-02-01-02-05-00.png,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,RAVDESS/spectograms/03-01-03-02-02-01-19-00.png,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,RAVDESS/spectograms/03-01-02-01-02-01-15-00.png,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
3,RAVDESS/spectograms/03-01-08-01-01-02-05-02.png,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,RAVDESS/spectograms/03-01-05-02-02-01-01-02.png,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# Export to Google Drive

This part is optional. It is used to export the modified Data Set to your Google Drive.

In [13]:
!zip -r RAVDESS_train.zip RAVDESS/split_image

  adding: RAVDESS/split_image/ (stored 0%)
  adding: RAVDESS/split_image/03-01-03-02-02-02-24-00-500.png (deflated 52%)
  adding: RAVDESS/split_image/03-01-07-01-01-01-22-00-2500.png (deflated 53%)
  adding: RAVDESS/split_image/03-01-02-02-02-02-15-01-3500.png (deflated 51%)
  adding: RAVDESS/split_image/03-01-01-01-01-01-05-02-1500.png (deflated 52%)
  adding: RAVDESS/split_image/03-01-08-02-01-02-02-02-0.png (deflated 54%)
  adding: RAVDESS/split_image/03-01-04-02-02-01-03-00-4000.png (deflated 53%)
  adding: RAVDESS/split_image/03-01-05-02-01-01-21-01-2000.png (deflated 52%)
  adding: RAVDESS/split_image/03-01-07-02-01-01-06-02-500.png (deflated 53%)
  adding: RAVDESS/split_image/03-01-03-01-01-01-08-01-1000.png (deflated 54%)
  adding: RAVDESS/split_image/03-01-07-02-02-02-15-01-5000.png (deflated 52%)
  adding: RAVDESS/split_image/03-01-02-01-01-02-15-01-1000.png (deflated 53%)
  adding: RAVDESS/split_image/03-01-04-02-01-01-22-02-3500.png (deflated 52%)
  adding: RAVDESS/split_im

In [14]:
!ls -l

total 1095800
-rw-r--r-- 1 root root      2634 Nov 18 14:29 adc.json
drwxr-xr-x 8 root root      4096 Nov 18 13:20 RAVDESS
-rw-r--r-- 1 root root 904686601 Nov 18 14:32 RAVDESS_spectogram.zip
-rw-r--r-- 1 root root 216990518 Nov 18 14:52 RAVDESS_train.zip
drwxr-xr-x 1 root root      4096 Nov  6 16:17 sample_data
-rw-r--r-- 1 root root     25590 Nov 18 12:04 testcrop0.png
-rw-r--r-- 1 root root     25920 Nov 18 12:04 testcrop1000.png
-rw-r--r-- 1 root root     28581 Nov 18 11:05 testcrop1300.png
-rw-r--r-- 1 root root     26135 Nov 18 12:04 testcrop1500.png
-rw-r--r-- 1 root root     25748 Nov 18 12:04 testcrop2000.png
-rw-r--r-- 1 root root     26336 Nov 18 12:04 testcrop2500.png
-rw-r--r-- 1 root root     28068 Nov 18 11:34 testcrop2795.png
-rw-r--r-- 1 root root     26000 Nov 18 12:04 testcrop3000.png
-rw-r--r-- 1 root root     26199 Nov 18 12:04 testcrop3500.png
-rw-r--r-- 1 root root     26070 Nov 18 12:04 testcrop4000.png
-rw-r--r-- 1 root root     26188 Nov 18 12:04 testcrop4500.

In [15]:
# Export the data to your drive

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# zip name must match the name given to zip
zip_name = 'RAVDESS_train.zip'

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': zip_name})
uploaded.SetContentFile(zip_name)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1N0zUWDYfZR4xUDQVNKJEnO9bFL4Y1NRM


In [0]:

# split the images into even pieces
def split_image_data(width = 500):
  im = Image.open("/content/RAVDESS/spectograms/03-01-03-01-02-02-13-01.png")
  imgwidth, imgheight = im.size
  for j in range(0,imgwidth,width):
      if j+width > imgwidth:
        j = imgwidth - width
      box = (j, 0, j+width, imgheight)
      a = im.crop(box)
      a.save('/content/testcrop' + str(j) + '.png')


split_image_data()